### Ques1

In [3]:
"""We’ll solve this by:

Finding tree centers using the leaf deletion algorithm.

Rooting the trees at the centers (if there are 2 centers, we try both).

Applying AHU canonical encoding to compare structure."""

'We’ll solve this by:\n\nFinding tree centers using the leaf deletion algorithm.\n\nRooting the trees at the centers (if there are 2 centers, we try both).\n\nApplying AHU canonical encoding to compare structure.'

In [1]:
from collections import defaultdict, deque

def find_centers(adj):
    """
    Find the center(s) of a tree using the Leaf Deletion Algorithm.
    Returns a list with 1 or 2 centers.
    """
    n = len(adj)
    degree = {node: len(neigh) for node, neigh in adj.items()}
    leaves = deque([node for node in adj if degree[node] == 1])
    
    remaining_nodes = n
    while remaining_nodes > 2:
        leaf_count = len(leaves)
        remaining_nodes -= leaf_count
        
        for _ in range(leaf_count):
            leaf = leaves.popleft()
            degree[leaf] = 0  # removed
            for neighbor in adj[leaf]:
                degree[neighbor] -= 1
                if degree[neighbor] == 1:
                    leaves.append(neighbor)
    
    return list(leaves)


def ahu_encode(node, parent, adj):
    """
    Recursively encode the tree rooted at 'node' using AHU algorithm.
    Canonical form: "(" + sorted child codes + ")"
    """
    labels = []
    for neighbor in adj[node]:
        if neighbor != parent:
            labels.append(ahu_encode(neighbor, node, adj))
    labels.sort()
    return "(" + "".join(labels) + ")"


def canonical_form(adj):
    """
    Compute the canonical form of a tree by rooting it at its center(s).
    If two centers, try both and pick the smaller code.
    """
    centers = find_centers(adj)
    encodings = [ahu_encode(center, None, adj) for center in centers]
    return min(encodings)


def areIsomorphicUnrooted(tree1, tree2):
    """
    Check if two unrooted trees are isomorphic.
    Each tree is given as an adjacency list (dict of lists).
    """
    if len(tree1) != len(tree2):  # different number of nodes
        return False
    
    code1 = canonical_form(tree1)
    code2 = canonical_form(tree2)
    return code1 == code2

In [2]:
if __name__ == "__main__":
    # Example 1: Single center
    tree1 = {
        1: [2, 3],
        2: [1, 4, 5],
        3: [1],
        4: [2],
        5: [2]
    }
    tree2 = {
        "a": ["b", "c"],
        "b": ["a", "d", "e"],
        "c": ["a"],
        "d": ["b"],
        "e": ["b"]
    }
    print("Test 1 (single center):", areIsomorphicUnrooted(tree1, tree2))  # True

    # Example 2: Two centers (a line of 4 nodes)
    tree3 = {
        1: [2],
        2: [1, 3],
        3: [2, 4],
        4: [3]
    }
    tree4 = {
        "x": ["y"],
        "y": ["x", "z"],
        "z": ["y", "w"],
        "w": ["z"]
    }
    print("Test 2 (two centers):", areIsomorphicUnrooted(tree3, tree4))  # True

    # Example 3: Non-isomorphic
    tree5 = {
        1: [2, 3],
        2: [1],
        3: [1, 4],
        4: [3]
    }
    tree6 = {
        "a": ["b", "c"],
        "b": ["a"],
        "c": ["a", "d", "e"],
        "d": ["c"],
        "e": ["c"]
    }
    print("Test 3 (non-isomorphic):", areIsomorphicUnrooted(tree5, tree6))  # False

Test 1 (single center): True
Test 2 (two centers): True
Test 3 (non-isomorphic): False


### Ques 2

In [5]:
from collections import defaultdict

def max_fun_factor(n, weights, edges):
    """
    n       : number of employees (nodes)
    weights : dict {node: fun factor}
    edges   : list of (parent, child) relationships
    """

    # Step 1: Build adjacency list (children list)
    children = defaultdict(list)
    for parent, child in edges:
        children[parent].append(child)

    # Step 2: Post-order traversal using recursion
    M = {}  # DP table: M[v] = max fun factor for subtree rooted at v

    def dfs(v):
        # If leaf → its best is just its own weight
        if not children[v]:
            M[v] = weights[v]
            return M[v]

        # Case 1: Exclude v → take max fun factor of all children
        exclude_v = 0
        for u in children[v]:
            exclude_v += dfs(u)

        # Case 2: Include v → add v’s weight + all grandchildren’s M[]
        include_v = weights[v]
        for u in children[v]:
            for g in children[u]:  # grandchildren
                include_v += M[g]

        # Take maximum
        M[v] = max(exclude_v, include_v)
        return M[v]

    # Assume CEO is root (node 1)
    return dfs(1)

In [6]:
if __name__ == "__main__":
    # Sample Input
    n = 7
    weights = {
        1: 10,
        2: 5,
        3: 6,
        4: 4,
        5: 3,
        6: 7,
        7: 2
    }
    edges = [
        (1, 2), (1, 3),
        (2, 4), (2, 5),
        (3, 6), (3, 7)
    ]

    result = max_fun_factor(n, weights, edges)
    print("Maximum Fun Factor =", result)  # Expected: 26

    # Additional Test Case
    n2 = 5
    weights2 = {
        1: 8,
        2: 2,
        3: 9,
        4: 1,
        5: 10
    }
    edges2 = [
        (1, 2), (1, 3),
        (2, 4), (3, 5)
    ]
    # Explanation:
    # If CEO (8) attends → cannot take 2,3 but can take 4(1),5(10) → 8+1+10=19
    # If CEO does not attend → can take 2(2)+3(9)=11 (better is CEO case)
    # Answer = 19
    result2 = max_fun_factor(n2, weights2, edges2)
    print("Maximum Fun Factor (extra test) =", result2)  # Expected: 19

Maximum Fun Factor = 26
Maximum Fun Factor (extra test) = 19
